# AVALIAÇÃO 1 - PROGRAMAÇÃO EM FINANÇAS
#### Professor: Julio Russo
#### Disciplina: Programação em Finanças
#### Turma: 2025.1
#### Alunos: Lucas dos Santos Marques, Luiz Fernando M. Meirinho , Victor A. S. de Souza e Victor Flávio P. Dornelos
#### Data: 16/04/2025


# 1 - Introdução


Este projeto tem como objetivo responder às questões propostas na Avaliação 1 da disciplina **Programação em Finanças**. Para isso, o presente notebook reúne tanto a implementação dos códigos em Python quanto as análises correspondentes realizadas a partir dos resultados.

O desenvolvimento está sendo feito no **Visual Studio Code (VS Code)**, mas o código pode ser executado em qualquer IDE compatível com Python. Todas as bibliotecas utilizadas estão listadas no arquivo `requirements.txt`, e o projeto foi desenvolvido utilizando a versão **3.12.0 do Python**.

In [82]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [83]:
# Bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from IPython.display import display, Markdown
from tabulate import tabulate

# 2- Questão 1:

### 1) CDBs e Conversão de Taxas

Buscar ao menos **3 CDBs de bancos de sua preferência** que declarem **taxas ao ano** e transformá-las em **taxas capitalizadas mensalmente**, programando esse processo em Python.

Além disso, declarar qual será o **retorno líquido dos investimentos, já descontados impostos e taxas**, para cada uma das opções analisadas.


#### **Premissas Gerais**
- **Valor do Aporte Inicial:** R$10.000,00  
- **Período de Investimento:** 03/04/2024 a 30/03/2026 (24 meses)  
- **Plataforma:** Nubank – Perfil Cliente Normal  
- **Imposto de Renda (IR):** 17,5%  
- **Taxa de Administração:** 0% (isenta)

---

#### **CDBs Selecionados**

| Banco                | Taxa Nominal Anual (a.a) | Alíquota de IR (%) | Taxa de Administração (%) |
|----------------------|---------------------------|---------------------|----------------------------|
| **Banco Original**   | 15,08%                    | 17,5%               | 0%                         |
| **Banco ABC Brasil** | 14,64%                    | 17,5%               | 0%                         |
| **Banco Caixa Geral**| 14,62%                    | 17,5%               | 0%                         |


In [84]:
def analisar_cdbs_evolucao(cdbs, aporte_inicial, meses, ir_percentual):
    registros = []
    resultados_finais = []

    print("🔍 Análise de CDBs\n")

    for banco, taxa_aa in cdbs.items():
        # Taxa mensal composta
        taxa_am = (1 + taxa_aa) ** (1 / 12) - 1

        for t in range(1, meses + 1):
            montante = aporte_inicial * (1 + taxa_am) ** t
            juros = montante - aporte_inicial
            retorno_liquido = juros * (1 - ir_percentual)

            registros.append({
                "Mês": t,
                "Banco": banco,
                "Retorno Líquido (R$)": retorno_liquido
            })

        retorno_final = registros[-1]["Retorno Líquido (R$)"]
        resultados_finais.append({
            "Banco": banco,
            "Taxa Mensal (%)": round(taxa_am * 100, 4),
            "Retorno Líquido Final (R$)": round(retorno_final, 2)
        })

        # Print dos resultados do banco
        print(f"🏦 {banco}")
        print(f" - Taxa mensal: {taxa_am * 100:.4f}%")
        print(f" - Retorno líquido após {meses} meses: R$ {retorno_final:,.2f}\n")

    # DataFrame com evolução
    df_evolucao = pd.DataFrame(registros)

    # Gráfico
    fig = px.line(df_evolucao,
                  x="Mês",
                  y="Retorno Líquido (R$)",
                  color="Banco",
                  title="📈 Evolução do Retorno Líquido dos CDBs",
                  markers=True)
    
    fig.update_layout(height=500)
    fig.show()

    # Ranking final
    df_ranking = pd.DataFrame(resultados_finais)
    df_ranking = df_ranking.sort_values(by="Retorno Líquido Final (R$)", ascending=False).reset_index(drop=True)

    print("🏆 Ranking dos CDBs (maior para menor retorno líquido):\n")
    print(tabulate(df_ranking, headers='keys', tablefmt='grid', showindex=False))

In [85]:
cdbs = {
    "Banco Original": 0.1508,
    "Banco ABC Brasil": 0.1464,
    "Banco Caixa Geral": 0.1462
}

analisar_cdbs_evolucao(cdbs, aporte_inicial=10000, meses=24, ir_percentual=0.175)



🔍 Análise de CDBs

🏦 Banco Original
 - Taxa mensal: 1.1774%
 - Retorno líquido após 24 meses: R$ 2,675.81

🏦 Banco ABC Brasil
 - Taxa mensal: 1.1451%
 - Retorno líquido após 24 meses: R$ 2,592.42

🏦 Banco Caixa Geral
 - Taxa mensal: 1.1436%
 - Retorno líquido após 24 meses: R$ 2,588.64



🏆 Ranking dos CDBs (maior para menor retorno líquido):

+-------------------+-------------------+------------------------------+
| Banco             |   Taxa Mensal (%) |   Retorno Líquido Final (R$) |
+===================+===================+==============================+
| Banco Original    |            1.1774 |                      2675.81 |
+-------------------+-------------------+------------------------------+
| Banco ABC Brasil  |            1.1451 |                      2592.42 |
+-------------------+-------------------+------------------------------+
| Banco Caixa Geral |            1.1436 |                      2588.64 |
+-------------------+-------------------+------------------------------+


# 3 - Questão 2:

### 2) Comparação de Retorno: Tesouro IPCA+ vs. Juros Pré-fixados

Programar em Python uma simulação para comparar o **retorno líquido** entre o **Tesouro IPCA+** e um título com **juros pré-fixados**, considerando **tempos de vencimentos equivalentes**.

O valor do investimento monetário fica a **critério do grupo simular**.

O objetivo é verificar **qual dos dois investimentos se torna mais vantajoso após os descontos de impostos e taxas**.


## Premissas

- **Aplicação única**: R$10.000
- **Inflação estimada (IPCA)**: 4% ao ano
- **Imposto de Renda (IR)**: a definir conforme prazo (≥ 720 dias = 15%)
- **Prazo da aplicação**: até 01/01/2035
- **Data de aplicação**: 03/02/2025
- **Taxa de administração**: 0,2% ao ano
- **Pagamento de juros**: considerado apenas no vencimento (para simplificação)

---

## Títulos Públicos Analisados

| Título                          | Tipo de Rentabilidade       | Taxa de Juros              | Vencimento   | Fonte (Consulta)    |
|--------------------------------|-----------------------------|----------------------------|--------------|----------------------|
| Tesouro IPCA+ 2035             | Pós-fixado (IPCA + real)    | IPCA + 7,49% a.a           | 01/01/2035   | Tesouro Direto (03/02/2025) |
| Tesouro Prefixado 2035         | Pré-fixado                  | 14,76% a.a                 | 01/01/2035   | Tesouro Direto (03/02/2025) |

> **Observação**: Para fins de simplificação, considerou-se que o pagamento dos juros ocorre integralmente no vencimento final (sem cupons semestrais).



In [86]:
valor_investido = 10000  # Valor da aplicação única
prazo_meses = 119        # Aproximadamente de 03/02/2025 até 01/01/2035
prazo_anos = prazo_meses / 12

meses = np.arange(0, prazo_meses + 1)  # Vetor de 0 até 119 meses
anos = meses / 12  # Convertendo meses para frações de ano

# Taxas fornecidas
ipca_estimado = 0.04  # 4% ao ano
taxa_ipca_mais = ipca_estimado + 0.0749  # IPCA + 7,49% a.a
taxa_prefixado = 0.1476  # 14,76% a.a

# Imposto de renda para aplicações acima de 720 dias
ir = 0.15  # 15%
taxa_adm = 0.002  # 0,2% a.a

# Taxas líquidas efetivas
taxa_liquida_ipca = (1 + taxa_ipca_mais)**(1 - ir) - 1 - taxa_adm
taxa_liquida_prefixado = (1 + taxa_prefixado)**(1 - ir) - 1 - taxa_adm

# Cálculo do montante ao longo do tempo
montante_ipca = valor_investido * (1 + taxa_liquida_ipca) ** anos
montante_prefixado = valor_investido * (1 + taxa_liquida_prefixado) ** anos

# Valor final
valor_final_ipca = montante_ipca[-1]
valor_final_prefixado = montante_prefixado[-1]

# Retorno líquido
retorno_ipca = valor_final_ipca - valor_investido
retorno_prefixado = valor_final_prefixado - valor_investido

In [ ]:
df_plot = pd.DataFrame({
    'Meses': meses,
    'Tesouro IPCA+ 2035': montante_ipca,
    'Tesouro Prefixado 2035': montante_prefixado
})

df_melted = df_plot.melt(id_vars='Meses', var_name='Investimento', value_name='Valor')

fig = px.line(df_melted, x='Meses', y='Valor', color='Investimento',
              title='Evolução do Valor Investido ao Longo do Tempo',
              labels={'Valor': 'Valor Acumulado (R$)', 'Meses': 'Tempo (meses)'})
fig.show()

# TABELA DE RESUMO E RANKING
resumo = pd.DataFrame({
    "Investimento": ["Tesouro IPCA+ 2035", "Tesouro Prefixado 2035"],
    "Taxa Líquida (%)": [round(taxa_liquida_ipca * 100, 4), round(taxa_liquida_prefixado * 100, 4)],
    "Valor Final (R$)": [round(valor_final_ipca, 4), round(valor_final_prefixado, 4)],
    "Retorno Líquido (R$)": [round(retorno_ipca, 4), round(retorno_prefixado, 4)]
})

resumo["Ranking"] = resumo["Valor Final (R$)"].rank(ascending=False).astype(int)

# Exibir em formato tabulate para copiar como markdown
print(tabulate(resumo, headers='keys', tablefmt='grid', showindex=False))

+------------------------+--------------------+--------------------+------------------------+-----------+
| Investimento           |   Taxa Líquida (%) |   Valor Final (R$) |   Retorno Líquido (R$) |   Ranking |
+========================+====================+====================+========================+===========+
| Tesouro IPCA+ 2035     |             9.4858 |            24564   |                14564   |         1 |
+------------------------+--------------------+--------------------+------------------------+-----------+
| Tesouro Prefixado 2035 |            12.2144 |            31355.6 |                21355.6 |         2 |
+------------------------+--------------------+--------------------+------------------------+-----------+


# 4- Questão 3: 

### 3) Programar em Python o quanto de aporte financeiro é necessário para alcançar em 7 anos, 10 anos e 12 anos:

- A) R$30.000  
- B) R$50.000  
- C) R$100.000  

Considerando uma **taxa de juros real efetiva de 7% ao ano** em todos esses cenários.


In [88]:
def gerar_dados(montante_desejado, anos, taxa_juros_anual):
    i = (1 + taxa_juros_anual) ** (1 / 12) - 1
    n = anos * 12
    R = montante_desejado * i / ((1 + i) ** n - 1)
    meses = np.arange(1, n + 1)
    montantes = R * ((1 + i) ** meses - 1) / i
    df = pd.DataFrame({
        'Mês': meses,
        'Montante Acumulado (R$)': montantes,
        'Prazo (anos)': f'{anos} anos'
    })
    return R, df

In [89]:
def calcular_para_varios_anos_plotly(montante_desejado, anos_lista, taxa_juros_anual=0.07):
    resultados = {}
    dfs = []

    for anos in anos_lista:
        R, df = gerar_dados(montante_desejado, anos, taxa_juros_anual)
        resultados[anos] = R
        dfs.append(df)

        # Print dos aportes
        display(Markdown(f"**{anos} anos** → R$ {R:.2f} por mês"))

        # Gráfico individual com Plotly
        fig = px.line(df, x='Mês', y='Montante Acumulado (R$)',
                      title=f'Evolução do Montante - {anos} anos',
                      markers=True)
        fig.update_layout(template='plotly_white')
        fig.show()

    # Gráfico combinado com todos os anos
    df_total = pd.concat(dfs, ignore_index=True)
    fig_total = px.line(df_total, x='Mês', y='Montante Acumulado (R$)', color='Prazo (anos)',
                        title=f'Evolução do Montante para Meta de R$ {montante_desejado:,.2f} - Todos os Prazos',
                        markers=True)
    fig_total.update_layout(template='plotly_white')
    fig_total.show()

    return resultados

## A)

In [90]:
calcular_para_varios_anos_plotly(
    montante_desejado=30000,
    anos_lista=[7, 10, 12]
)



**7 anos** → R$ 280.01 por mês

**10 anos** → R$ 175.39 por mês

**12 anos** → R$ 135.46 por mês

{7: 280.0091587617208, 10: 175.38553851336093, 12: 135.46198770024267}

## B)

In [91]:
b = calcular_para_varios_anos_plotly(
    montante_desejado=50000,
    anos_lista=[7, 10, 12]
)

**7 anos** → R$ 466.68 por mês

**10 anos** → R$ 292.31 por mês

**12 anos** → R$ 225.77 por mês

## C)

In [92]:
c = calcular_para_varios_anos_plotly(
    montante_desejado=100000,
    anos_lista=[7, 10, 12]
)

**7 anos** → R$ 933.36 por mês

**10 anos** → R$ 584.62 por mês

**12 anos** → R$ 451.54 por mês